# word2vecを用いて、単語の分散表現を獲得する
## <メモリエラーが発生する場合は、小説を宮沢賢治だけにしてみてください>

In [2]:
import numpy as np
import pandas as pd

try:
    from google.colab import files
    print('Google Colab. 上での実行です')
    !pip install -U mecab-python3==0.996.2
    print('「ファイルを選択」から、1_dataフォルダの中身を全て選択し、アップロードしてください')
    print('===========')
    files.upload()
    %mkdir -p ../1_data
    !mv *.txt ../1_data
except:
    print('ローカル環境での実行です')

Google Colab. 上での実行です
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
「ファイルを選択」から、1_dataフォルダの中身を全て選択し、アップロードしてください


Saving gingatetsudono_yoru.txt to gingatetsudono_yoru.txt
Saving kaijin_nijumenso.txt to kaijin_nijumenso.txt
Saving wagahaiwa_nekodearu.txt to wagahaiwa_nekodearu.txt


### データの下準備
* まず、青空文庫のデータを読み込み、前処理を行う。
* データは多い方がいいが、今回は問題を簡単にするため、夏目漱石、宮沢賢治、江戸川乱歩の作品から１冊づつ選ぶことにする。

In [3]:
import re
import random

# 夏目漱石
wagahai_file = open("../1_data/wagahaiwa_nekodearu.txt", encoding = "utf-8")
wagahai = wagahai_file.read()
wagahai_file.close()

# 宮沢賢治
ginga_file = open("../1_data/gingatetsudono_yoru.txt", encoding = "utf-8")
ginga = ginga_file.read()
ginga_file.close()

# 江戸川乱歩
kaijin_file = open("../1_data/kaijin_nijumenso.txt", encoding = "utf-8")
kaijin = kaijin_file.read()
kaijin_file.close()

novels = {"Soseki":wagahai, "Kenji":ginga, "Ranpo": kaijin}
# novels = {"Kenji":ginga}

for author, novel in novels.items():
    novel = re.sub("《[^》]+》", "", novel) # ルビの削除
    novel = re.sub("［[^］]+］", "", novel) # 読みの注意の削除
    novel = re.sub("[｜ 　「」\n]", "", novel) # | と全角半角スペース、「」と改行の削除

    seperator = "。"
    sentence_list = novel.split(seperator)
    sentence_list.pop() # 最後の要素は空の文字列になるので、削除
    sentence_list = [x+seperator for x in sentence_list]
    
    novels[author] = sentence_list
    
##  文章数を揃えたい場合、以下のコメントアウトを戻す 
## ※ 文章数を揃えた場合，ランダムに文章を選択する関係上，「音」を含む行が削除され，後半のセルが実行できない場合があるため注意すること
# s_min = len(list(novels.values())[0])
# for novel in novels.values():
#     if len(novel) < s_min:
#         s_min = len(novel) 
# for key, novel in novels.items():
#     novels[key] = random.sample(novel, s_min) # 最も文章数が少ない小説に揃える

# print(novels["Soseki"])

### 分かち書き
* Mecabを用いて分かち書きを行う

In [4]:
import MeCab

tagger = MeCab.Tagger("-Owakati") 

spaced_novels = {}

for author, novel in novels.items():
    
    spaced_sentences = []
    
    for sentence in novel:
        spaced_sentence = tagger.parse(sentence)
        
        # 改行の削除
        spaced_sentence = re.sub("[\n]", "", spaced_sentence) 
        
        # タブ区切りのテキストをリストに変換する
        spaced_sentence = [i for i in spaced_sentence.split(" ") if not i in [",", "", " ", "、","。"] ]
        
        # リストに追加
        spaced_sentences.append(spaced_sentence)
        
    spaced_novels[author] = spaced_sentences

spaced_novels["All"] = spaced_novels["Soseki"] + spaced_novels["Kenji"] + spaced_novels["Ranpo"] 
# spaced_novels["All"] = spaced_novels["Kenji"] 
# print(spaced_novels["Soseki"])

### 単語の出現頻度
* 単語の出現頻度を数える

In [5]:
from collections import Counter

data = spaced_novels["Soseki"]
# data = spaced_novels["Kenji"]
data = [i for sent in data for i in sent]

counter = Counter(data)
for word, cnt in counter.most_common():
    print (word, cnt )

ストリーミング出力は最後の 5000 行に切り捨てられました。
恩顧 1
除い 1
畏服 1
天が下 1
一隅 1
名乗っ 1
取扱い 1
燻り 1
大島紬 1
古渡 1
おまけ 1
へええ 1
あっ気 1
縁辺 1
魂消 1
苦々しく 1
気転 1
利かす 1
伺えれ 1
ゃるんでやりたいなんてえんじゃ 1
参らせる 1
ゃしませんそれじゃ 1
喧嘩腰 1
軍配 1
土俵際 1
持ち直す 1
反り返る 1
行司 1
気取り 1
乗り出す 1
からまっ 1
狐付き 1
感じ入る 1
阿部 1
演奏 1
嵌っ 1
有れ 1
撃 1
呆然 1
瘧 1
箍 1
ゆるん 1
持前 1
本態 1
復する 1
崩れる 1
睨みつける 1
ウフン 1
なりゃ 1
陳述 1
なさろ 1
おっしゃろ 1
構っ 1
わるけりゃ 1
憚り様 1
御門 1
枴仙 1
軍鶏 1
蹴合い 1
思い付い 1
救い 1
根堀 1
あらい 1
荒立て 1
磁気 1
怪訝 1
いただく 1
成れ 1
関する 1
渋い 1
併せ 1
天体 1
運行 1
話題 1
楊子 1
填め 1
引掛 1
填める 1
絵葉書 1
旧暦 1
御山 1
スッポコポンノポン 1
羽衣 1
一心に 1
星 1
微妙 1
沁む 1
寒さ 1
聞き惚れ 1
倶楽部 1
さんざん 1
やられる 1
三枚目 1
印刷 1
書き散らし 1
よべ 1
泊り 1
荒磯 1
さよの 1
船乗り 1
乗りゃ 1
得手 1
見送り 1
クツクツ 1
口気 1
一気 1
中央 1
剋 1
売れ残っ 1
讒訴 1
アイソクラチス 1
ソフォクリス 1
高齢 1
シモニジス 1
着せ 1
御召 1
逃 1
連綿 1
生き延び 1
半々 1
オホホホホホ 1
しま 1
被れ 1
被る 1
庶境 1
克己 1
此年 1
寄こし 1
送れ 1
問い返し 1
祝捷 1
間に合う 1
調達 1
仕立てる 1
間に合っ 1
かおっ 1
当日 1
翁 1
棺 1
大間 1
小包 1
遣わし 1
小為替 1
此方 1
送 1
漢学 1
聖堂 1
朱子学 1
こり 1
固まっ 1
やたら 1
アハハハハハ 1
局長 1
単なる 1
曰く付き 1
術数 1
天来 1
コメディー 1
立ち至り 1
俯目 1
無関係 1
権勢 1
浮べ 1
買収 1
ニヤニヤ 1
したて 

### word2vecを用いた学習
* ニューラルネットワークの学習を行う

In [ ]:
from gensim.models import word2vec

models = {}
for author, data in spaced_novels.items():
    
    print(author, "文章数=%s"%len(data))

    # word2vecモデルの学習
    # size : 中間層のノード数
    # min_count : この値以下の頻度の単語を無視
    # window : 対象単語を中心とした前後の単語数
    # iter : epochs数
    # sg : skip-gramを使うかどうか 0:CBOW 1:skip-gram
    model = word2vec.Word2Vec(data,
                              size=100,
                              min_count=5,
                              window=5,
                              iter=100,
                             sg = 0)
    models[author] = model

Soseki 文章数=7486
Kenji 文章数=1120
Ranpo 文章数=3164
All 文章数=11770


### 類似度の高い単語の出力
* 学習データが少ないため、あまりおもしろい結果にはならない

In [ ]:
for author, model  in models.items():
    print(author)
    results = model.wv.most_similar(positive=['音']) # 3つの小説のいずれにも出現する単語を指定する。夜、時間など
#     print([(p,round(q,3))for p,q in results])
    display(pd.DataFrame(results, columns=["単語", "類似度"]).round(3))

Soseki


,単語,類似度
0,声,0.564
1,我慢,0.357
2,甕,0.350
3,心持,0.343
4,左衛門,0.334
5,縁,0.333
6,ステッキ,0.326
7,軽く,0.321
8,わす,0.320
9,足音,0.320


Kenji


,単語,類似度
0,すきとおっ,0.705
1,かたち,0.634
2,流れ,0.570
3,波,0.569
4,なり,0.557
5,しずか,0.552
6,向う岸,0.547
7,川下,0.545
8,聞え,0.540
9,声,0.536


Ranpo


,単語,類似度
0,ソッ,0.549
1,黒い,0.544
2,はげしい,0.530
3,ドア,0.523
4,ガラス,0.513
5,大きな,0.493
6,なか,0.480
7,階段,0.465
8,ほお,0.461
9,ひらき,0.449


All


,単語,類似度
0,声,0.473
1,ガラガラ,0.376
2,きらっ,0.367
3,ぼり,0.367
4,条,0.365
5,足音,0.364
6,風,0.360
7,川,0.355
8,扉,0.352
9,縁,0.352


### ベクトル間の距離をコサイン類似度で測定

In [ ]:
vec1 = models["Soseki"].wv.__getitem__("音")
vec1

array([-1.581021  , -0.6310641 ,  0.08665089,  1.0494071 ,  1.9850612 ,
       -0.32968545, -1.0148636 , -1.325419  , -1.0499854 , -0.85646886,
        3.2385986 ,  3.026674  , -1.4540067 , -2.2028613 ,  1.7969604 ,
       -1.9295291 ,  1.2809653 ,  0.6206645 ,  0.22872004,  0.2543639 ,
        0.3786289 ,  0.90729666,  1.199147  ,  2.1951945 , -0.14010912,
        1.6284474 ,  2.500654  , -0.68467134, -2.6517239 ,  2.3989642 ,
       -0.35238472,  1.5991951 , -0.6793127 ,  0.37385452, -0.84432   ,
       -1.5814759 , -2.0976224 ,  3.8685567 ,  0.10062467, -0.20729037,
       -3.5651798 , -0.18107958,  1.2607654 ,  0.31083983,  3.9245112 ,
       -0.21335314, -1.6192554 ,  0.96305   , -1.7432965 ,  1.890575  ,
       -2.2923675 ,  2.3924832 , -0.29212603,  1.3335563 ,  1.0936012 ,
       -0.583023  , -0.9595855 , -0.44743943,  0.10741734, -0.37364453,
       -0.67505866, -0.10578997, -0.58230233, -1.6248574 ,  1.0158477 ,
       -0.8211131 ,  0.38779286,  1.2694666 ,  0.23859912,  2.37

In [ ]:
vec2 = models["Soseki"].wv.__getitem__("声")
vec2

array([ 0.5558855 ,  0.864212  ,  0.91488713,  3.0692575 ,  2.3057456 ,
        0.14871383, -0.97807765, -0.01357795, -0.8412504 , -2.029407  ,
        3.5075963 ,  4.8858624 , -1.4310858 , -0.8988453 ,  3.0348022 ,
       -1.9590627 ,  0.8491058 ,  0.64590615,  1.2000605 ,  2.1539533 ,
       -0.37620404,  0.77925014,  0.475378  ,  0.4795174 , -2.4724467 ,
       -0.40428817,  3.1379602 ,  0.5211068 ,  0.5036958 ,  1.8241674 ,
        1.5898397 ,  1.2248547 , -1.6481625 , -1.7342311 , -1.6122929 ,
        0.30068898,  0.9216357 ,  0.14788435,  0.84039253,  0.6262741 ,
       -1.6784868 , -0.05423421, -0.8532581 , -1.355563  ,  3.5691493 ,
        2.2347498 , -3.8376772 , -0.9979489 ,  1.5384672 ,  0.88505155,
       -1.4578971 , -1.5118747 , -0.9328044 , -1.9060229 ,  2.797515  ,
       -1.5312499 , -0.7856641 , -3.5237148 , -1.0370551 , -0.85360795,
        0.39277145,  0.01048527, -4.3662143 ,  0.29912272,  0.44705623,
       -2.4348938 , -1.5182275 ,  1.6422242 ,  1.0694206 ,  2.70

In [ ]:
# コサイン類似度を算出
# model.wv.most_similar()で算出した類似度と一致することがわかる
np.dot(vec1, vec2) / (np.linalg.norm(vec1)*np.linalg.norm(vec2))

0.56364214

### 単語ベクトルの演算
* 学習データが少ないため、あまりおもしろい結果にはならない
* 良い学習モデルが完成すれば、 フランス　-　パリ + 東京 = 日本 のような演算が可能になる

In [ ]:
# 人間 - 夢
models["All"].wv.most_similar(positive=['人間'], negative=["夢"])

[('出来ん', 0.3673709034919739),
 ('格別', 0.36153534054756165),
 ('困難', 0.35465627908706665),
 ('足り', 0.35458073019981384),
 ('完全', 0.35413306951522827),
 ('起す', 0.3532378077507019),
 ('罪', 0.33426564931869507),
 ('愛', 0.324072003364563),
 ('力学', 0.32166367769241333),
 ('積極', 0.31915995478630066)]

### 分散表現の確認

In [ ]:
model = models["All"]
print(model.wv.vectors.shape)
print("重みW_in=")
print(model.wv.vectors)
print("これが分散表現")

(4397, 100)
重みW_in=
[[-0.03359391 -0.9894963  -0.76144934 ...  0.2349094   0.12697764
   1.1641201 ]
 [ 0.69779456 -0.27393395 -0.29497075 ...  2.303921   -0.3375219
   1.5092503 ]
 [ 0.6491809  -0.46612838 -0.4695462  ...  0.7182078   0.41141143
   0.7875898 ]
 ...
 [ 0.44982344 -1.5278487  -0.06093991 ... -0.5155084  -0.01065704
   0.41754237]
 [-0.9584538  -0.72956586  0.4370155  ... -0.53280073 -0.62664694
   0.22270858]
 [-0.30427277 -1.8841095   0.59842294 ...  0.13323122 -0.29264522
   0.1968402 ]]
これが分散表現


In [ ]:
print("語彙の数=%s"% len(model.wv.vocab.keys()))
model.wv.vocab.keys()

語彙の数=4397


dict_keys(['吾輩', 'は', '猫', 'で', 'ある', '名前', 'まだ', '無い', 'どこ', '生れ', 'た', 'か', 'とんと', '見当', 'が', 'つか', 'ぬ', '何', 'でも', 'し', '所', '泣い', 'て', 'だけ', '記憶', 'いる', 'ここ', '始め', '人間', 'という', 'もの', 'を', '見', 'しかも', 'あと', '聞く', 'と', 'それ', '書生', '中', '一番', 'な', 'あっ', 'そう', 'だ', 'この', 'の', '時々', '我々', '捕え', '煮', '食う', '話', 'しかし', 'その', '当時', '考', 'も', 'なかっ', 'から', '別段', '思わ', 'ただ', '彼', 'に', '載せ', 'られ', '時', '何だか', '感じ', 'ばかり', '上', '少し', '落ちつい', '顔', 'いわゆる', 'あろ', 'う', '妙', '思っ', '今', '残っ', '第', '一', '毛', 'をもって', '装飾', 'さ', 'れ', 'べき', 'はず', 'つるつる', 'まるで', '薬缶', 'その後', 'だいぶ', '逢っ', 'こんな', '片', '輪', '度', '事', 'ない', 'のみ', 'なら', 'ず', '真中', 'あまりに', 'そうして', '穴', '煙', '吹く', 'どうも', '咽', 'せ', '実に', '弱っ', 'これ', '飲む', '煙草', 'ようやく', '頃', '知っ', '裏', 'しばらく', 'よい', '心持', '坐っ', 'おっ', 'する', '非常', '運転', '動く', '自分', '分ら', '無', '暗に', '眼', '廻る', '胸', '悪く', 'なる', '到底', 'ど', 'さり', '音', '火', '出', 'まで', 'やら', 'いくら', 'として', 'ふと', '気', '付い', '見る', 'い', 'たくさん', '一疋', '見え', '肝心', 'さえ', '姿', '隠し', 'しまっ', '違っ', '明るい', 'くらい', 'は

In [ ]:
# 語彙の順番
model.wv.index2word[:10]

['の', 'て', 'に', 'は', 'を', 'が', 'た', 'と', 'で', 'も']

In [ ]:
# 1番目の語彙の単語ベクトル
model.wv.vectors[0]

array([-0.03359391, -0.9894963 , -0.76144934, -0.02398266,  0.97484535,
        0.38206747,  0.51945424, -0.27857754, -0.53238297, -0.07423376,
       -0.15907706,  0.9775074 , -0.41739044, -0.01959823,  0.24325864,
        0.47793397, -0.2602978 ,  0.353801  ,  0.07586347,  0.2548854 ,
        0.3532192 , -0.76422745, -0.5307285 ,  0.15711226,  0.37821966,
       -1.0212023 ,  0.33386675,  0.0719863 ,  1.1085695 , -0.22704849,
       -0.1473707 ,  1.4948844 ,  0.54621756,  0.1542936 , -0.7499564 ,
        0.27167314,  0.03980968, -0.0166867 , -0.2979652 ,  0.6716737 ,
        0.43668908,  0.05079527,  1.055301  , -0.25584084,  0.65119   ,
       -0.49492398, -0.12058429, -0.26176292, -0.2414082 , -0.35518923,
        0.34391928, -0.8046122 , -0.00223006,  0.36819774, -0.9731601 ,
        0.2947431 , -1.1826984 , -0.7975819 ,  0.22347952,  0.6458227 ,
       -0.2411678 , -0.9767414 , -0.2747075 ,  0.7965506 ,  0.650483  ,
        0.1493823 , -0.00992364, -0.05938024,  0.6336627 ,  0.56

In [ ]:
# 1番目の語彙の単語ベクトル
model.wv.__getitem__("の")

array([-0.03359391, -0.9894963 , -0.76144934, -0.02398266,  0.97484535,
        0.38206747,  0.51945424, -0.27857754, -0.53238297, -0.07423376,
       -0.15907706,  0.9775074 , -0.41739044, -0.01959823,  0.24325864,
        0.47793397, -0.2602978 ,  0.353801  ,  0.07586347,  0.2548854 ,
        0.3532192 , -0.76422745, -0.5307285 ,  0.15711226,  0.37821966,
       -1.0212023 ,  0.33386675,  0.0719863 ,  1.1085695 , -0.22704849,
       -0.1473707 ,  1.4948844 ,  0.54621756,  0.1542936 , -0.7499564 ,
        0.27167314,  0.03980968, -0.0166867 , -0.2979652 ,  0.6716737 ,
        0.43668908,  0.05079527,  1.055301  , -0.25584084,  0.65119   ,
       -0.49492398, -0.12058429, -0.26176292, -0.2414082 , -0.35518923,
        0.34391928, -0.8046122 , -0.00223006,  0.36819774, -0.9731601 ,
        0.2947431 , -1.1826984 , -0.7975819 ,  0.22347952,  0.6458227 ,
       -0.2411678 , -0.9767414 , -0.2747075 ,  0.7965506 ,  0.650483  ,
        0.1493823 , -0.00992364, -0.05938024,  0.6336627 ,  0.56

一致していることがわかる

### [演習]
* 学習用データを増やして、word2vecの学習をさせてみましょう